In [1]:
import sys
import os
import glob
import re

import pandas as pd
import pandas_profiling as pdp
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')

In [16]:
df = pd.read_csv('../processed_data/train_v2.csv')
df.head(1)

,id,y,location,access,layout,age,direction,area,floor,bath_toilet,...,enviroment,structure,contact_period,max_floor,room_num,L,D,K,S,north
0,1,75000,東京都北区滝野川３丁目,都営三田線\t西巣鴨駅\t徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線\...,1K,117,南東,20.01,1,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座,...,【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...,7,2年間,12,1,0,0,1,0,0


In [17]:
df.columns

Index(['id', 'y', 'location', 'access', 'layout', 'age', 'direction', 'area',
       'floor', 'bath_toilet', 'kitchen', 'broadcast_com', 'facilities',
       'parking', 'enviroment', 'structure', 'contact_period', 'max_floor',
       'room_num', 'L', 'D', 'K', 'S', 'north'],
      dtype='object')

In [18]:
df = df.loc[:,['id', 'age','area','floor', 'structure', 'max_floor',
       'room_num', 'L', 'D', 'K', 'S', 'north', 'y']]
df.head()

,id,age,area,floor,structure,max_floor,room_num,L,D,K,S,north,y
0,1,117,20.01,1,7,12,1,0,0,1,0,0,75000
1,2,538,16.50,5,2,10,1,0,0,0,0,0,76000
2,3,102,22.05,12,7,15,1,0,0,1,0,0,110000
3,4,352,60.48,3,7,4,2,1,1,1,0,0,150000
4,5,379,39.66,1,1,2,2,0,1,1,0,0,74000


In [19]:
X, y = df.loc[:,'age':'north'], df['y']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=0)

In [21]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [22]:
params = {
        'task' : 'train',
        'boosting_type' : 'gbdt',
        'objective' : 'regression',
        'metric' : 'rmse',
}

In [23]:
model = lgb.train(params, lgb_train, valid_sets=lgb_eval,num_boost_round=5000)

[1]	valid_0's rmse: 69064.9
[2]	valid_0's rmse: 64770.7
[3]	valid_0's rmse: 60897.6
[4]	valid_0's rmse: 57673.1
[5]	valid_0's rmse: 54800.9
[6]	valid_0's rmse: 52354.3
[7]	valid_0's rmse: 50142.7
[8]	valid_0's rmse: 48257.1
[9]	valid_0's rmse: 46627.4
[10]	valid_0's rmse: 45221.5
[11]	valid_0's rmse: 44015.6
[12]	valid_0's rmse: 43033.5
[13]	valid_0's rmse: 42191.1
[14]	valid_0's rmse: 41407.4
[15]	valid_0's rmse: 40733
[16]	valid_0's rmse: 40217.8
[17]	valid_0's rmse: 39793.1
[18]	valid_0's rmse: 39332.4
[19]	valid_0's rmse: 39000.4
[20]	valid_0's rmse: 38721
[21]	valid_0's rmse: 38396.4
[22]	valid_0's rmse: 38180.7
[23]	valid_0's rmse: 38000.5
[24]	valid_0's rmse: 37835.3
[25]	valid_0's rmse: 37602.4
[26]	valid_0's rmse: 37450.6
[27]	valid_0's rmse: 37309.9
[28]	valid_0's rmse: 37214.6
[29]	valid_0's rmse: 37135.7
[30]	valid_0's rmse: 37040.4
[31]	valid_0's rmse: 36951.6
[32]	valid_0's rmse: 36914.6
[33]	valid_0's rmse: 36826.4
[34]	valid_0's rmse: 36736.6
[35]	valid_0's rmse: 36670.

[498]	valid_0's rmse: 32473.8
[499]	valid_0's rmse: 32451.9
[500]	valid_0's rmse: 32447.6
[501]	valid_0's rmse: 32449.5
[502]	valid_0's rmse: 32461.7
[503]	valid_0's rmse: 32460.6
[504]	valid_0's rmse: 32455
[505]	valid_0's rmse: 32454.5
[506]	valid_0's rmse: 32441.3
[507]	valid_0's rmse: 32440.7
[508]	valid_0's rmse: 32428.8
[509]	valid_0's rmse: 32421.7
[510]	valid_0's rmse: 32418.4
[511]	valid_0's rmse: 32418.2
[512]	valid_0's rmse: 32421
[513]	valid_0's rmse: 32410.7
[514]	valid_0's rmse: 32403.9
[515]	valid_0's rmse: 32417.6
[516]	valid_0's rmse: 32410.7
[517]	valid_0's rmse: 32402.3
[518]	valid_0's rmse: 32398.5
[519]	valid_0's rmse: 32392.7
[520]	valid_0's rmse: 32396.4
[521]	valid_0's rmse: 32393.2
[522]	valid_0's rmse: 32413.8
[523]	valid_0's rmse: 32409.1
[524]	valid_0's rmse: 32394.1
[525]	valid_0's rmse: 32392.6
[526]	valid_0's rmse: 32382.9
[527]	valid_0's rmse: 32379.6
[528]	valid_0's rmse: 32396.2
[529]	valid_0's rmse: 32370.6
[530]	valid_0's rmse: 32361.4
[531]	valid_0'

[1021]	valid_0's rmse: 31817.2
[1022]	valid_0's rmse: 31812.6
[1023]	valid_0's rmse: 31811.9
[1024]	valid_0's rmse: 31810.1
[1025]	valid_0's rmse: 31804.9
[1026]	valid_0's rmse: 31815
[1027]	valid_0's rmse: 31814.5
[1028]	valid_0's rmse: 31811.9
[1029]	valid_0's rmse: 31811.8
[1030]	valid_0's rmse: 31813.8
[1031]	valid_0's rmse: 31811.4
[1032]	valid_0's rmse: 31809.2
[1033]	valid_0's rmse: 31796.1
[1034]	valid_0's rmse: 31791.6
[1035]	valid_0's rmse: 31793.4
[1036]	valid_0's rmse: 31790.2
[1037]	valid_0's rmse: 31792.8
[1038]	valid_0's rmse: 31792.4
[1039]	valid_0's rmse: 31790.4
[1040]	valid_0's rmse: 31798.8
[1041]	valid_0's rmse: 31798.3
[1042]	valid_0's rmse: 31798.5
[1043]	valid_0's rmse: 31795.9
[1044]	valid_0's rmse: 31795.9
[1045]	valid_0's rmse: 31795.1
[1046]	valid_0's rmse: 31794
[1047]	valid_0's rmse: 31795.9
[1048]	valid_0's rmse: 31793.9
[1049]	valid_0's rmse: 31793.5
[1050]	valid_0's rmse: 31788.4
[1051]	valid_0's rmse: 31784.7
[1052]	valid_0's rmse: 31781.6
[1053]	valid

[1527]	valid_0's rmse: 31590.4
[1528]	valid_0's rmse: 31589.8
[1529]	valid_0's rmse: 31599.6
[1530]	valid_0's rmse: 31599.1
[1531]	valid_0's rmse: 31598.5
[1532]	valid_0's rmse: 31598.5
[1533]	valid_0's rmse: 31598.5
[1534]	valid_0's rmse: 31602.7
[1535]	valid_0's rmse: 31598.2
[1536]	valid_0's rmse: 31600.8
[1537]	valid_0's rmse: 31601.5
[1538]	valid_0's rmse: 31600.8
[1539]	valid_0's rmse: 31603.2
[1540]	valid_0's rmse: 31602.3
[1541]	valid_0's rmse: 31601.5
[1542]	valid_0's rmse: 31602.4
[1543]	valid_0's rmse: 31601.5
[1544]	valid_0's rmse: 31601.5
[1545]	valid_0's rmse: 31601.8
[1546]	valid_0's rmse: 31601.6
[1547]	valid_0's rmse: 31601.7
[1548]	valid_0's rmse: 31602.3
[1549]	valid_0's rmse: 31602.5
[1550]	valid_0's rmse: 31602.8
[1551]	valid_0's rmse: 31604.4
[1552]	valid_0's rmse: 31607.6
[1553]	valid_0's rmse: 31604.3
[1554]	valid_0's rmse: 31603.1
[1555]	valid_0's rmse: 31602.8
[1556]	valid_0's rmse: 31606.6
[1557]	valid_0's rmse: 31604.9
[1558]	valid_0's rmse: 31609.7
[1559]	v

[1989]	valid_0's rmse: 31556.9
[1990]	valid_0's rmse: 31555.2
[1991]	valid_0's rmse: 31555.1
[1992]	valid_0's rmse: 31555.4
[1993]	valid_0's rmse: 31555
[1994]	valid_0's rmse: 31554.5
[1995]	valid_0's rmse: 31555.2
[1996]	valid_0's rmse: 31554.7
[1997]	valid_0's rmse: 31553.6
[1998]	valid_0's rmse: 31553.7
[1999]	valid_0's rmse: 31554.8
[2000]	valid_0's rmse: 31556.2
[2001]	valid_0's rmse: 31556.1
[2002]	valid_0's rmse: 31553.6
[2003]	valid_0's rmse: 31557.4
[2004]	valid_0's rmse: 31555.1
[2005]	valid_0's rmse: 31558.5
[2006]	valid_0's rmse: 31558.5
[2007]	valid_0's rmse: 31558.1
[2008]	valid_0's rmse: 31555.8
[2009]	valid_0's rmse: 31555.6
[2010]	valid_0's rmse: 31555.6
[2011]	valid_0's rmse: 31555.1
[2012]	valid_0's rmse: 31554.3
[2013]	valid_0's rmse: 31552.5
[2014]	valid_0's rmse: 31552.1
[2015]	valid_0's rmse: 31552.5
[2016]	valid_0's rmse: 31552
[2017]	valid_0's rmse: 31551.8
[2018]	valid_0's rmse: 31550.8
[2019]	valid_0's rmse: 31550.2
[2020]	valid_0's rmse: 31550.2
[2021]	valid

...

[3272]	valid_0's rmse: 31521.7
[3273]	valid_0's rmse: 31521.6
[3274]	valid_0's rmse: 31519.7
[3275]	valid_0's rmse: 31520.1
[3276]	valid_0's rmse: 31518.7
[3277]	valid_0's rmse: 31514
[3278]	valid_0's rmse: 31514
[3279]	valid_0's rmse: 31513.1
[3280]	valid_0's rmse: 31513.4
[3281]	valid_0's rmse: 31513.1
[3282]	valid_0's rmse: 31513
[3283]	valid_0's rmse: 31512.8
[3284]	valid_0's rmse: 31512.6
[3285]	valid_0's rmse: 31512.2
[3286]	valid_0's rmse: 31512.1
[3287]	valid_0's rmse: 31509.8
[3288]	valid_0's rmse: 31509.1
[3289]	valid_0's rmse: 31511.9
[3290]	valid_0's rmse: 31510.8
[3291]	valid_0's rmse: 31510.1
[3292]	valid_0's rmse: 31508.2
[3293]	valid_0's rmse: 31507.7
[3294]	valid_0's rmse: 31506.9
[3295]	valid_0's rmse: 31506.8
[3296]	valid_0's rmse: 31505
[3297]	valid_0's rmse: 31505.7
[3298]	valid_0's rmse: 31506.9
[3299]	valid_0's rmse: 31506.7
[3300]	valid_0's rmse: 31506.8
[3301]	valid_0's rmse: 31506.9
[3302]	valid_0's rmse: 31507.5
[3303]	valid_0's rmse: 31506.2
[3304]	valid_0's

[3545]	valid_0's rmse: 31544.1
[3546]	valid_0's rmse: 31544.6
[3547]	valid_0's rmse: 31544.7
[3548]	valid_0's rmse: 31544.5
[3549]	valid_0's rmse: 31545.2
[3550]	valid_0's rmse: 31543
[3551]	valid_0's rmse: 31542.1
[3552]	valid_0's rmse: 31540.7
[3553]	valid_0's rmse: 31540.9
[3554]	valid_0's rmse: 31540.6
[3555]	valid_0's rmse: 31542.3
[3556]	valid_0's rmse: 31541.4
[3557]	valid_0's rmse: 31541.9
[3558]	valid_0's rmse: 31540.7
[3559]	valid_0's rmse: 31541.7
[3560]	valid_0's rmse: 31541.5
[3561]	valid_0's rmse: 31540.8
[3562]	valid_0's rmse: 31540.6
[3563]	valid_0's rmse: 31540.9
[3564]	valid_0's rmse: 31541
[3565]	valid_0's rmse: 31540
[3566]	valid_0's rmse: 31540.5
[3567]	valid_0's rmse: 31541.3
[3568]	valid_0's rmse: 31541.3
[3569]	valid_0's rmse: 31541.4
[3570]	valid_0's rmse: 31541.2
[3571]	valid_0's rmse: 31540.6
[3572]	valid_0's rmse: 31542.2
[3573]	valid_0's rmse: 31542.5
[3574]	valid_0's rmse: 31541.6
[3575]	valid_0's rmse: 31542
[3576]	valid_0's rmse: 31541.8
[3577]	valid_0's

[4076]	valid_0's rmse: 31542.9
[4077]	valid_0's rmse: 31544.1
[4078]	valid_0's rmse: 31543.2
[4079]	valid_0's rmse: 31543.4
[4080]	valid_0's rmse: 31543.6
[4081]	valid_0's rmse: 31543.5
[4082]	valid_0's rmse: 31543.7
[4083]	valid_0's rmse: 31543.1
[4084]	valid_0's rmse: 31543.5
[4085]	valid_0's rmse: 31543.6
[4086]	valid_0's rmse: 31543.6
[4087]	valid_0's rmse: 31543
[4088]	valid_0's rmse: 31542.7
[4089]	valid_0's rmse: 31543.5
[4090]	valid_0's rmse: 31544.2
[4091]	valid_0's rmse: 31541.4
[4092]	valid_0's rmse: 31542.2
[4093]	valid_0's rmse: 31542.1
[4094]	valid_0's rmse: 31542
[4095]	valid_0's rmse: 31543.3
[4096]	valid_0's rmse: 31543.3
[4097]	valid_0's rmse: 31543.8
[4098]	valid_0's rmse: 31545.6
[4099]	valid_0's rmse: 31544.6
[4100]	valid_0's rmse: 31544.2
[4101]	valid_0's rmse: 31543.9
[4102]	valid_0's rmse: 31544.2
[4103]	valid_0's rmse: 31543.9
[4104]	valid_0's rmse: 31544.3
[4105]	valid_0's rmse: 31543.6
[4106]	valid_0's rmse: 31543.3
[4107]	valid_0's rmse: 31540.1
[4108]	valid

[4571]	valid_0's rmse: 31551
[4572]	valid_0's rmse: 31551
[4573]	valid_0's rmse: 31550.6
[4574]	valid_0's rmse: 31550.7
[4575]	valid_0's rmse: 31550.8
[4576]	valid_0's rmse: 31550.4
[4577]	valid_0's rmse: 31549.9
[4578]	valid_0's rmse: 31550
[4579]	valid_0's rmse: 31550.3
[4580]	valid_0's rmse: 31550.4
[4581]	valid_0's rmse: 31550.2
[4582]	valid_0's rmse: 31550.2
[4583]	valid_0's rmse: 31550
[4584]	valid_0's rmse: 31548.1
[4585]	valid_0's rmse: 31548.5
[4586]	valid_0's rmse: 31548.5
[4587]	valid_0's rmse: 31549.1
[4588]	valid_0's rmse: 31549.2
[4589]	valid_0's rmse: 31549.1
[4590]	valid_0's rmse: 31548.8
[4591]	valid_0's rmse: 31548.7
[4592]	valid_0's rmse: 31549.6
[4593]	valid_0's rmse: 31549.3
[4594]	valid_0's rmse: 31548.5
[4595]	valid_0's rmse: 31548.8
[4596]	valid_0's rmse: 31548.7
[4597]	valid_0's rmse: 31548.7
[4598]	valid_0's rmse: 31548.4
[4599]	valid_0's rmse: 31548.5
[4600]	valid_0's rmse: 31548.5
[4601]	valid_0's rmse: 31548.6
[4602]	valid_0's rmse: 31548.6
[4603]	valid_0's

[4886]	valid_0's rmse: 31550.7
[4887]	valid_0's rmse: 31551
[4888]	valid_0's rmse: 31551
[4889]	valid_0's rmse: 31550.6
[4890]	valid_0's rmse: 31550.5
[4891]	valid_0's rmse: 31550.4
[4892]	valid_0's rmse: 31550.6
[4893]	valid_0's rmse: 31550.8
[4894]	valid_0's rmse: 31550
[4895]	valid_0's rmse: 31550.8
[4896]	valid_0's rmse: 31551.2
[4897]	valid_0's rmse: 31551.1
[4898]	valid_0's rmse: 31551.4
[4899]	valid_0's rmse: 31551.9
[4900]	valid_0's rmse: 31551.7
[4901]	valid_0's rmse: 31551.8
[4902]	valid_0's rmse: 31552.1
[4903]	valid_0's rmse: 31552.3
[4904]	valid_0's rmse: 31552.4
[4905]	valid_0's rmse: 31551.2
[4906]	valid_0's rmse: 31552.4
[4907]	valid_0's rmse: 31552.3
[4908]	valid_0's rmse: 31551.3
[4909]	valid_0's rmse: 31551.7
[4910]	valid_0's rmse: 31551
[4911]	valid_0's rmse: 31550.9
[4912]	valid_0's rmse: 31549.9
[4913]	valid_0's rmse: 31549.7
[4914]	valid_0's rmse: 31550.3
[4915]	valid_0's rmse: 31551
[4916]	valid_0's rmse: 31551.2
[4917]	valid_0's rmse: 31550.6
[4918]	valid_0's r

In [26]:
test_df = pd.read_csv('../processed_data/test_v2.csv')
test_df = test_df.loc[:,['id', 'age','area','floor', 'structure', 'max_floor',
       'room_num', 'L', 'D', 'K', 'S', 'north']]
test_df.head()

,id,age,area,floor,structure,max_floor,room_num,L,D,K,S,north
0,31471,588,50.22,8,2.0,8,2,1,1,1,0,0
1,31472,2,20.88,3,7.0,4,1,0,0,0,0,0
2,31473,280,26.93,1,2.0,4,1,0,0,1,0,0
3,31474,434,23.57,1,1.0,2,1,0,0,1,0,0
4,31475,544,50.00,4,7.0,4,2,1,1,1,0,0


In [27]:
y_pred = model.predict(test_df, num_iteration=model.best_iteration)

In [28]:
y_pred

array([282173.69576978,  75546.73238216, 474548.02806264, ...,
        36879.95172601, 248303.48033399, 289135.50801266])

In [29]:
submit_file = pd.concat([test_df['id'],pd.Series(y_pred)],axis=1)
submit_file

,id,0
0,31471,282173.695770
1,31472,75546.732382
2,31473,474548.028063
3,31474,479381.950963
4,31475,232468.680013
...,...,...
31257,62728,45040.025206
31258,62729,458804.237530
31259,62730,36879.951726
31260,62731,248303.480334


In [30]:
submit_file.to_csv('../submit/0927_submit.csv',header=False,index=False)